## Задание

Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/ Ниже примерный список того, что можно попробовать в ходе решения.

1. Проведите предобработку текста: уберите небуквенные символы, проведите токенизацию, лемматизируйте текст. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic_label).В качестве данных для классификации в пунктах 3 и 5 попробуйте взять
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
2. Обучите fastText для классификации текстов по темам (baseline).
3. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
4. Обучите на полученных средних векторах алгоритм классификации (логистическая регрессия, SVM, KNN, однослойную полносвязную нейросеть). 
5. Обучите сверточную сеть, на вход можно подавать предобученные эмбеддинги из п. 4 **(для слов, не для предложений)**.

In [1]:
! wget -O lenta-ru-train.csv https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0

--2020-04-16 20:45:08--  https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:1::a27d:4601, 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:1::a27d:4601|:443... failed: Network is unreachable.
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv [following]
--2020-04-16 20:45:08--  https://www.dropbox.com/s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6ed18f9e8398cebcb8c300156.dl.dropboxusercontent.com/cd/0/inline/A19CcALyr0NY0zr5SCry1vT7OozwhnrMygGFDzmsMNqfLxaxHFedmxSmXbHJAEwz5ARGcD46aTzxYDWuClfPYMWjPnOAB6_xHATZeIt2K_Ce6xAkiTBEFuW-saZ6rSL8Sao/file# [following]
--2020-04-16 20:45:08--  https://uce6ed18f9e8398cebcb8c300156.dl.dropboxu

In [45]:
import pandas as pd

lenta = pd.read_csv('lenta-ru-train.csv')
lenta.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


In [46]:
lenta.topic.value_counts()

Экономика          63571
Спорт              51679
Культура           43072
Наука и техника    42384
Бизнес              5931
Name: topic, dtype: int64

In [47]:
len(lenta)

206637

Ниже пример, как можно сделать файл с решением для Kaggle.

In [4]:
! wget -O lenta-ru-test.csv https://www.dropbox.com/s/9av90tgqob1slnj/lenta-ru-test.csv?dl=0

--2020-04-17 21:11:52--  https://www.dropbox.com/s/9av90tgqob1slnj/lenta-ru-test.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:1::a27d:4601, 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:1::a27d:4601|:443... failed: Network is unreachable.
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9av90tgqob1slnj/lenta-ru-test.csv [following]
--2020-04-17 21:11:52--  https://www.dropbox.com/s/raw/9av90tgqob1slnj/lenta-ru-test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaa82188e4d98928094f4a60e9d.dl.dropboxusercontent.com/cd/0/inline/A2CQ_IxeHDlZnv6KWIIGMMBbur627hfdbsHN-vFTX6_aOhPGSOlboh7e5qzwNEbFMpOmcIlb_9zqpvNs-gHrVhCTxZywfJzk-cFunkiNFc2MCmfwqmS7kcXFynijMxE5qXY/file# [following]
--2020-04-17 21:11:53--  https://ucaa82188e4d98928094f4a60e9d.dl.dropboxuser

In [48]:
from random import randint

topics = ['Экономика', 'Спорт', 'Культура', 'Наука и техника', 'Бизнес']
test_data = pd.read_csv('lenta-ru-test.csv')
test_data.head()

,title,text
0,Сборная России выиграла чемпионат мира по футб...,В южноафриканском Кейптауне победой сборной Ро...
1,Анна Курникова снова проиграла в первом раунде,"Анна Курникова, проиграв испанке Кончите Марти..."
2,"Новым главным тренером ""Спартака"" стал Станисл...",Новым главным тренером московского футбольного...
3,Майкл Дуглас совершил неожиданный визит на Кубу,Известный голливудский актер Майкл Дуглас сове...
4,Кадыров снялся в короткометражке «Волшебный гр...,Глава Чеченской Республики Рамзан Кадыров снял...


In [49]:
lenta['title'][0]

'Инфляция в\xa0январе 2006\xa0года составит 2,6\xa0процента'

In [50]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('snowball_data')
nltk.download('perluniprops')
nltk.download('universal_tagset')
nltk.download('stopwords')
nltk.download('nonbreaking_prefixes')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/care1e55/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package nonbreakin

True

In [51]:
# !pip install pymystem3

In [52]:
import re
import unicodedata
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from functools import lru_cache

In [53]:
sw = stopwords.words('russian')

In [54]:
lenta['title'] = lenta['title'].str.replace(r'[^А-я\s]', '', regex=True).str.lower().str.split()
lenta['text'] = lenta['text'].str.replace(r'[^А-я\s]', '', regex=True).str.lower().str.split()

In [55]:
lenta['text']

0         [глава, росстата, владимир, соколин, заявил, ч...
1         [у, российских, кинематографистов, появится, н...
2         [медицинский, штаб, миланского, интера, обнаро...
3         [премии, книга, года, в, номинациях, вручены, ...
4         [сборная, португалии, сыграла, вничью, с, кома...
                                ...                        
206632    [бывшей, вицепрезиденту, ювелирной, компании, ...
206633    [минфин, предлагает, увеличить, размер, штрафа...
206634    [индекс, потребительской, уверенности, россиян...
206635    [капитан, женской, сборной, россии, по, теннис...
206636    [союз, европейских, футбольных, ассоциаций, уе...
Name: text, Length: 206637, dtype: object

In [56]:
lenta[lenta['text'].apply(lambda x: print(x) if type(x) is float else 'Nope') != 'Nope']

nan


,title,text,topic,topic_label
151352,"[гребенщиков, заработал, тысяч, рублей, в, омс...",NaN,Культура,3


In [57]:
lenta = lenta.dropna()

In [58]:
def clean(l):
    new_l = [w for w in l if w not in sw]
    return new_l

In [73]:
lenta['title'] = lenta['title'].apply(clean)
lenta['text'] = lenta['text'].apply(clean)

In [77]:
lenta.head()

,title,text,topic,topic_label
0,"(инфляция, январь, год, составлять, процент)","(глава, росстата, владимир, соколин, заявил, я...",Экономика,0
1,"(никита, михалков, учреждать, день, российский...","(российских, кинематографистов, появится, новы...",Культура,3
2,"(марко, матерацци, вернуться, строй, матч, цска)","(медицинский, штаб, миланского, интера, обнаро...",Спорт,1
3,"(определять, лауреат, премия, книга, год)","(премии, книга, года, номинациях, вручены, мос...",Культура,3
4,"(гол, роналд, штрафной, спасать, португалец, п...","(сборная, португалии, сыграла, вничью, командо...",Спорт,1


In [76]:
lenta['title'] = lenta['title'].apply(tuple)
lenta['text'] = lenta['text'].apply(tuple)

In [78]:
lemmatizer = Mystem()

In [79]:
@lru_cache(maxsize=128)
def lemm(l):
    new_l = [lemmatizer.lemmatize(word)[0] for word in l]
    return new_l

In [ ]:
lenta['title'] = lenta['title'].apply(lemm)
lenta['text'] = lenta['text'].apply(lemm)

In [ ]:
lenta.head()

In [ ]:
def cat(l):
    return " ".join(l)

In [ ]:
lenta['title'] = lenta.title.apply(cat)
lenta['text'] = lenta.text.apply(cat)

In [ ]:
lenta.to_csv('lenta_clean.csv')

In [ ]:
import pickle

In [ ]:
lemmfile = open('lemmpickle', 'ab')
pickle.dump(lenta, lemmfile)
lemmfile.close() 

In [ ]:
lemmfileloaded = open('lemmpickle', 'rb')      
lenta_loaded = pickle.load(lemmfileloaded)

In [ ]:
lenta_loaded.head()

In [7]:
lenta = pd.read_csv('lenta_clean.csv')

In [ ]:
# ! pip install fastText/.

In [21]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

In [37]:
from fasttext import train_supervised

In [38]:
import fasttext

In [23]:
lenta.title[0]

['инфляция', 'январь', 'год', 'составлять', 'процент']

In [25]:
lenta.title.apply(cat)

0                    инфляция январь год составлять процент
1            никита михалков учреждать день российский кино
2                 марко матерацци вернуться строй матч цска
3                       определять лауреат премия книга год
4         гол роналд штрафной спасать португалец поражен...
                                ...                        
206632    бывший вицепрезидент обвинять миллионный кража...
206633    минфин предлагать увеличивать штраф незаконный...
206634    потребительский уверенность россиянин вернутьс...
206635                    тарпищев давать американский виза
206636    уефа пригрозить россия исключение число участн...
Name: title, Length: 206636, dtype: object

In [26]:
X = lenta.title.apply(cat).tolist()
y = lenta.topic.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 138446
total test examples 68190


In [27]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [39]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])s
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.9514884880481009
R@1: 0.9514884880481009
Number of examples: 68190


In [7]:
test_data['label'] = [topics[randint(0,4)] for i in range(len(test_data)) ]
test_data.head()

,title,text,label
0,Сборная России выиграла чемпионат мира по футб...,В южноафриканском Кейптауне победой сборной Ро...,Экономика
1,Анна Курникова снова проиграла в первом раунде,"Анна Курникова, проиграв испанке Кончите Марти...",Спорт
2,"Новым главным тренером ""Спартака"" стал Станисл...",Новым главным тренером московского футбольного...,Спорт
3,Майкл Дуглас совершил неожиданный визит на Кубу,Известный голливудский актер Майкл Дуглас сове...,Экономика
4,Кадыров снялся в короткометражке «Волшебный гр...,Глава Чеченской Республики Рамзан Кадыров снял...,Культура


In [8]:
test_data['label'].to_csv('submission.csv', header = None)